In [1]:
import pandas as pd
import numpy as np
import yfinance, ta

# Buy Low Strategy
This strategy taking the newest low in n periods as a trigger with Stochastic RSI oversold.

### Part 1: Buying Requirements
  1. Stochastic RSI K and D line must be in the oversold area (> n%)
  2. Stochastic RSI K line is below D line
  3. The stock made an n period low during this period of Stochastic RSI oversold
  4. Buy at close or at n% above new low

### Part 2: Selling Requirements
  1. Set Trailling stop to n% of high prices
  2. Set sell limit to a static % gain of buy price

## Getting Price Data
Getting the price data of symbol using the yfinance library.

In [2]:
# Loading the 10 years prices data from yfinance
# start: 2010-01-01 end: 2020-12-31
# Symbol: AAPL
symbol = 'AAPL'
ticker = yfinance.Ticker(symbol)
prices = ticker.history(start='2010-01-01', end='2020-12-31') 

In [3]:
prices

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-01-04,6.508334,6.540962,6.476315,6.526020,493729600,0.0,0.0
2010-01-05,6.544016,6.574205,6.502848,6.537307,601904800,0.0,0.0
2010-01-06,6.537303,6.563223,6.426609,6.433318,552160000,0.0,0.0
2010-01-07,6.457104,6.464727,6.374770,6.421425,477131200,0.0,0.0
2010-01-08,6.412888,6.464728,6.375076,6.464118,447610800,0.0,0.0
...,...,...,...,...,...,...,...
2020-12-23,130.808976,131.076205,129.443079,129.621246,88223700,0.0,0.0
2020-12-24,129.977531,132.095653,129.759779,130.620880,54930100,0.0,0.0
2020-12-28,132.620268,135.936013,132.145164,135.292664,124486200,0.0,0.0


## Technical Analysis
Using the ta library to do technical analysis.

In [4]:
# Create Stochastic RSI K
stoch_rsi = ta.momentum.StochRSIIndicator(prices.Close, window=14)
stoch_k = stoch_rsi.stochrsi_k()
stoch_d = stoch_rsi.stochrsi_d()
stoch_k

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2020-12-23    0.846730
2020-12-24    0.909331
2020-12-28    0.909331
2020-12-29    0.880770
2020-12-30    0.762838
Name: stochrsi_k, Length: 2768, dtype: float64

## Finding the Buy Signal

In [47]:
# Finding lows for 200 period, 100 period, 50 period and 20 period
period_range = [200, 100, 50, 20, 10]
low = prices.Low
# Simple function to find the lowest low in n periods
def get_lowest_low(s, length):
  return s.rolling(length).min()
# Getting a list of lows in different periods
lows = [get_lowest_low(low, _) for _ in period_range]
# Turn the list into a pandas DataFrame with concat
lows = pd.concat(lows, axis=1)
lows.columns = [str(_) + '_day_low' for _ in period_range]
# Check out lows
lows

,200_day_low,100_day_low,50_day_low,20_day_low,10_day_low
Date,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN,NaN
2010-01-06,NaN,NaN,NaN,NaN,NaN
2010-01-07,NaN,NaN,NaN,NaN,NaN
2010-01-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-23,52.282479,101.870286,106.039954,113.992648,118.921733
2020-12-24,52.282479,101.870286,106.039954,115.031912,119.317652
2020-12-28,52.282479,101.870286,106.039954,115.615878,120.297520


In [48]:
# Concat Low price with Stochastic RSI K and different period lows
columns = ['Low', 'StochRSI_K', 'StochRSI_D'] + lows.columns.tolist()
ndf = pd.concat([prices.Low, stoch_k, stoch_d, lows], axis=1)
# Dropna, with inplace=True so do have to reassign to the same variable
ndf.dropna(inplace=True)
# check the ndf 
ndf

,Low,stochrsi_k,stochrsi_d,200_day_low,100_day_low,50_day_low,20_day_low,10_day_low
Date,,,,,,,,
2010-10-18,9.583958,0.946677,0.843436,5.801483,7.183167,7.183167,8.385851,8.593818
2010-10-19,9.148811,0.834482,0.876421,5.801483,7.183167,7.183167,8.385851,8.698716
2010-10-20,9.357695,0.676624,0.819261,5.801483,7.183167,7.183167,8.385851,8.749033
2010-10-21,9.355557,0.500251,0.670452,5.801483,7.183167,7.183167,8.385851,8.843257
2010-10-22,9.340311,0.332435,0.503103,5.801483,7.183167,7.183167,8.385851,8.919189
...,...,...,...,...,...,...,...,...
2020-12-23,129.443079,0.846730,0.778968,52.282479,101.870286,106.039954,113.992648,118.921733
2020-12-24,129.759779,0.909331,0.833716,52.282479,101.870286,106.039954,115.031912,119.317652
2020-12-28,132.145164,0.909331,0.888464,52.282479,101.870286,106.039954,115.615878,120.297520


This is the logic for getting the buy signal for back test.

In [49]:
# Applying the strategy:
# Check if Stochastic RSI K is lower than 20% (.2), 
# Stochastic RSI D is lower than 20% (.2) and 
# Stochastic RSI K less than Stochastic RSI D
below = .25
low_stoch_rsi = (ndf.stochrsi_k < below) &  (ndf.stochrsi_d < below) & (ndf.stochrsi_k < ndf.stochrsi_d)
# Check if the price is one of the lows
# Using numpy's check to aviod pandas error
is_new_low = np.array(ndf.iloc[:, 2:]) >= np.array(ndf.loc[:, ['Low']])
# Check lowest is lower than more than n periods
is_new_low = is_new_low.sum(1) > 0
print('There are {} new period low in the dataframe of {} period.'.format(
  is_new_low.sum(), len(ndf)))

There are 330 new period low in the dataframe of 2569 period.


In [50]:
# Check for days of possible buys 
# If the Stochastic RSI K is lower then 20% and has an new low
has_hit = pd.concat([low_stoch_rsi, 
                    pd.Series(is_new_low, index=ndf.index)], 
                   axis=1).sum(1) > 1
signals = np.arange(len(has_hit))[has_hit]
print('There are {} signals.'.format(len(signals)))

There are 148 signals.


## Back Test

In [51]:
# Backtest loop
buy_at = 0
stop_limit = 0
stop_limit_pct = 1 - .12
sell_limit = 0
sell_limit_pct = 2
current_idx = 0
wins = []
num_of_trades = 0
for s in signals:
  # Continue when current index is greater s
  # was still holding
  if current_idx > s:
    continue
  # When there is a signal buy at close
  # getting the next price data and setting the stop limit and sell limit
  data = prices.iloc[s:, :4]
  buy_at = prices.Close.iloc[s]
  current_high = prices.High.iloc[s]
  stop_limit = current_high * stop_limit_pct
  sell_limit = buy_at * sell_limit_pct
  for i, (_, row) in enumerate(data.iloc[1:].iterrows()):
    i = i + 1
    # check sell conditions
    is_sell = False
    if row.Open < stop_limit:
      # if open prices is less than stop limit it's a sell
      wp = row.Open / buy_at - 1
      is_sell = True
    elif row.Low < stop_limit:
      # if low prices is less than stop limit, stop limit has been triggered
      wp = stop_limit / buy_at - 1
      is_sell = True
    elif row.High > sell_limit:
      # Sell when sell limit is reached
      wp = sell_limit / buy_at - 1
      is_sell = True
    
    if is_sell:
      # Selling logics
      wins.append(wp)
      current_idx = s + i
      num_of_trades += 1
      break

    # Update stop limit when there are new highs
    if row.High > current_high:
      stop_limit = row.High * stop_limit_pct
      current_high = row.High  

## Statistics

In [52]:
# Checking the strategy gain
strategy_gain = np.prod(np.array(wins) + 1) - 1
print(f'This Strategy has a gaining rate of : {strategy_gain*100:.2f}%')
print(f'Total trades of {num_of_trades}.') 

This Strategy has a gaining rate of : 773.68%
Total trades of 22.


In [53]:
# Seeing how much can be gain through moment
static_gain = prices.Close[-1] / prices.Close[0] - 1
print(f'The static gain of {symbol} is :  {static_gain*100:.2f}%') 

The static gain of AAPL is :  1928.08%


In [54]:
# Seeing how much can be gain when sell at the highest price
highest_gain = prices.High.max() / prices.Close[0] - 1
print(f'The maximum gain of {symbol} is :  {highest_gain*100:.2f}%') 

The maximum gain of AAPL is :  2004.98%


As you can see this strategy has an advantage to the base strategy, but we can even make it more profitable with some modify to certain parameters. 

The parameters are:
 - below: for setting or adjusting which is the oversold area.
 - stop_limit_pct: for setting the risk level for either a small or larger gain.
 - sell_limit_pct; for a set percentage gain